In [ ]:
import sys
import serial
import time
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QPushButton, QDoubleSpinBox,
    QMessageBox, QAbstractSpinBox
)
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QPixmap, QFont
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from PyQt5.QtGui import QIcon

# --- Sentetik veri üretme ve model değerlendirme ---
def generate_synthetic_data(n_samples=1000, k_true=0.0001, noise_std=0.02):
    np.random.seed(42)
    start_thickness = np.random.uniform(25.0, 35.0, size=n_samples)
    temperature = np.random.uniform(20.0, 100.0, size=n_samples)
    elapsed_time = np.random.uniform(0, 3600, size=n_samples)
    delta_h = k_true * temperature * elapsed_time
    true_thickness = start_thickness - delta_h
    true_thickness += np.random.normal(0, noise_std, size=n_samples)
    true_thickness = np.maximum(true_thickness, 0.0)
    df = pd.DataFrame({
        'start_thickness': start_thickness,
        'temperature': temperature,
        'elapsed_time': elapsed_time,
        'thickness': true_thickness
    })
    return df

# Üretilen veriyi değerlendir
os.makedirs('data', exist_ok=True)
df_synth = generate_synthetic_data()
X = (df_synth['temperature'] * df_synth['elapsed_time']).values.reshape(-1,1)
y = (df_synth['start_thickness'] - df_synth['thickness']).values
lr = LinearRegression(fit_intercept=False)
lr.fit(X, y)
k_est = lr.coef_[0]
y_pred = lr.predict(X)
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Sentetik veri üzerinde öğrenilen k değeri: {k_est:.6e}")
print(f"Model MAE: {mae:.4f} cm, RMSE: {rmse:.4f} cm")
# CSV olarak kaydet
synth_path = 'synthetic_data.csv'
df_synth.to_csv(synth_path, index=False)
print(f"Sentetik veri CSV olarak kaydedildi: {synth_path}")

class YuzeyAsinmaAsistani(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("YüzeyAşınmaAsistanı")
        self.setWindowIcon(QIcon("icon.png")) 
        self.setFixedSize(960, 540)

        # Ortak font
        self.ui_font = QFont("HK Grotesk", 12)

        # Arka plan
        bg_path = os.path.join(os.getcwd(), 'background.png')
        if os.path.exists(bg_path):
            bg_label = QLabel(self)
            pixmap = QPixmap(bg_path).scaled(960, 540)
            bg_label.setPixmap(pixmap)
            bg_label.setGeometry(0, 0, 960, 540)
            bg_label.lower()

        # Seri port ve timer
        self.port = "COM8"
        self.baud = 9600
        self.ser = None
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.veri_oku)

        # Model parametreleri
        self.ref_kalinlik = 0.0        # kullanıcı başlangıç kalınlığı
        self.kritik_esik = 0.0         # kullanıcı kritik kalınlık eşiği
        self.asinma_katsayisi = 0.0001 # linear katsayı
        self.start_time = None
        self.start_dist = None         # mesafe kalibrasyon değeri

        # Log dosyası
        os.makedirs("data", exist_ok=True)
        self.log_file = "data/logs.csv"
        with open(self.log_file, 'w', encoding='utf-8') as f:
            f.write("timestamp,temperature,distance,thickness,alarm\n")

        self.init_ui()

    def init_ui(self):
        # Başlangıç kalınlık
        self.spin_start = QDoubleSpinBox(self)
        self.spin_start.setRange(1.0, 100.0)
        self.spin_start.setDecimals(2)
        self.spin_start.setValue(30.0)
        self.spin_start.setButtonSymbols(QAbstractSpinBox.NoButtons)
        self.spin_start.setFont(self.ui_font)
        self.spin_start.setGeometry(200, 95, 120, 30)
        self.spin_start.setStyleSheet("background:transparent; color:white; border:none;")

        # Kritik eşik
        self.spin_threshold = QDoubleSpinBox(self)
        self.spin_threshold.setRange(1.0, 100.0)
        self.spin_threshold.setDecimals(2)
        self.spin_threshold.setValue(10.0)
        self.spin_threshold.setButtonSymbols(QAbstractSpinBox.NoButtons)
        self.spin_threshold.setFont(self.ui_font)
        self.spin_threshold.setGeometry(200, 140, 120, 30)
        self.spin_threshold.setStyleSheet("background:transparent; color:white; border:none;")

        # Sensör verisi etiketleri

        lbl_style = "color:white; background:transparent;"
        self.label_temp   = QLabel("", self)  # sıcaklık
        self.label_temp.setFont(self.ui_font)
        self.label_temp.setStyleSheet(lbl_style)
        self.label_temp.setGeometry(275,480,300,30)

        self.label_dist   = QLabel("", self) # mesafe
        self.label_dist.setFont(self.ui_font)
        self.label_dist.setStyleSheet(lbl_style)
        self.label_dist.setGeometry(450,480,300,30)

        self.label_thick  = QLabel("", self) # kalınlık
        self.label_thick.setFont(self.ui_font)
        self.label_thick.setStyleSheet(lbl_style)
        self.label_thick.setGeometry(630,480,300,30)

        self.label_alarm  = QLabel("", self) # durum
        self.label_alarm.setFont(self.ui_font)
        self.label_alarm.setStyleSheet(lbl_style)
        self.label_alarm.setGeometry(750,140,300,30)

        self.label_tahmin = QLabel("", self)
        self.label_tahmin.setFont(self.ui_font)
        self.label_tahmin.setStyleSheet(lbl_style)
        self.label_tahmin.setGeometry(750,95,600,30)

        # Başlat & Durdur butonları
        self.btn_baslat = QPushButton("", self)
        self.btn_baslat.setFont(self.ui_font)
        self.btn_baslat.setGeometry(50,330,140,40)
        self.btn_baslat.setStyleSheet("background:rgba(255,255,255,0); color:white;")
        self.btn_baslat.clicked.connect(self.baslat)

        self.btn_durdur = QPushButton("", self)
        self.btn_durdur.setFont(self.ui_font)
        self.btn_durdur.setGeometry(70,380,140,40)
        self.btn_durdur.setStyleSheet("background:rgba(255,255,255,0); color:white;")
        self.btn_durdur.setEnabled(False)
        self.btn_durdur.clicked.connect(self.durdur)
        
        # Grafik alanı (başta gizli)
        self.fig, self.ax = plt.subplots(facecolor='none')
        plt.close(self.fig)
        self.canvas = FigureCanvas(self.fig)
        self.canvas.setParent(self)
        self.canvas.setGeometry(270, 150, 500, 250)
        self.canvas.setStyleSheet("background:transparent;")
        self.canvas.hide()

        self.show()

    def baslat(self):
        # Referans ayarlama
        self.ref_kalinlik = self.spin_start.value()
        self.kritik_esik  = self.spin_threshold.value()

        try:
            if not self.ser:
                self.ser = serial.Serial(self.port, self.baud, timeout=1)
                time.sleep(2)
        except Exception as e:
            QMessageBox.critical(self, "Hata", f"Port açılamadı: {e}")
            return

        # Arayüzü kilitle
        self.spin_start.setEnabled(False)
        self.spin_threshold.setEnabled(False)
        self.btn_baslat.setEnabled(False)
        self.btn_durdur.setEnabled(True)

        # Sayaç başlat
        if not self.start_time:
            self.start_time = time.time()
        self.timer.start(1000)

    def durdur(self):
        self.timer.stop()
        if self.ser:
            self.ser.close()
            self.ser = None
        self.spin_start.setEnabled(True)
        self.spin_threshold.setEnabled(True)
        self.btn_baslat.setEnabled(True)
        self.btn_durdur.setEnabled(False)

    def veri_oku(self):
        line = self.ser.readline().decode('utf-8').strip()
        if not line:
            return
        try:
            parts = line.split('|')
            temp = float(parts[0].split(':')[1].replace('°C','').strip())
            dist = float(parts[1].split(':')[1].replace('cm','').strip())
        except:
            return

        # İlk mesafe kalibrasyonu
        if self.start_dist is None:
            self.start_dist = dist

        elapsed = time.time() - self.start_time

        # Duvar kalınlığı hesap
        thickness = self.ref_kalinlik - (dist - self.start_dist)
        thickness = max(thickness, 0.0)

        # Alarm
        alarm = 'Kritik' if thickness <= self.kritik_esik else 'Normal'

        # Etiket güncelle
        self.label_temp.setText(f"{temp:.1f} °C")
        self.label_dist.setText(f"{dist:.1f} cm")
        self.label_thick.setText(f"{thickness:.2f} cm")
        self.label_alarm.setText(alarm)

        # Tahmin
        if thickness <= self.kritik_esik:
            self.label_tahmin.setText("Zaten kritik seviyede")
        else:
            if temp > 0:
                t_crit = (thickness - self.kritik_esik) / (self.asinma_katsayisi * temp)
                t_crit = max(t_crit, 0.0)
                hedef = datetime.now() + timedelta(seconds=t_crit)
                self.label_tahmin.setText(f"{hedef.strftime('%H:%M:%S')} ({t_crit/60:.1f} dk)")

        # Log ve grafik
        with open(self.log_file, 'a', encoding='utf-8') as f:
            ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            f.write(f"{ts},{temp},{dist},{thickness:.2f},{alarm}\n")

        if self.canvas.isHidden():
            self.canvas.show()
        df = pd.read_csv(self.log_file)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['elapsed'] = (df['timestamp'] - df['timestamp'].iloc[0]).dt.total_seconds()
        self.ax.clear()
        self.ax.plot(df['elapsed'], df['thickness'], marker='o', color='white')
        self.ax.set_facecolor('none')
        self.fig.patch.set_alpha(0)
        self.ax.grid(color='gray')
        self.canvas.draw()

if __name__ == '__main__':
    app = QApplication.instance() or QApplication(sys.argv)
    window = YuzeyAsinmaAsistani()
    window.show()
    app.exec_()
